# Voiceshell Demo

#### Imports

In [3]:
import math
import random
from __future__ import unicode_literals
import spacy
import os
import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import wave, sys, pyaudio

#### Creating/defining basic functions

In [4]:
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0
    
def sentvec(s):
    sent = nlp(s)
    word_vectors = [w.vector for w in sent]
    return meanv(word_vectors)

def spacy_closest_sent(space, input_str, n=1):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                  reverse=True)[:n]

#### Set corpus file here:

In [5]:
nlp = spacy.load('en')

corpusFile = 'revisedCorpus/voiceshell_audio_LUT.csv'

In [7]:
with open(corpusFile, 'r') as f:
    reader = csv.reader(f)
    all_lines = list(reader)
    
audio_lookup_table = {}
sentences = []
for line in all_lines:
    sentence = line[0]
    if not sentence.isspace():
        sentences.append(nlp(sentence))
    filename = line[1]
    audio_lookup_table[sentence] = filename

In [8]:
prompts = ["How are you feeling?", "Tell me a quote you like.", "What are the first three words that pop into your head?", "What day is it today?", "What's your favourite colour?", "Where are you right now?", "What's your job?", "What do you like to eat?", "What animal would you want as a pet?"]

In [14]:
def runLoop():
    prompt = random.choice(prompts)
    print('Enter some text, anything at all...')
    userInput = input()
    
    for sent in spacy_closest_sent(sentences, userInput):
        print(sent.text)
    
    audioFile = audio_lookup_table[sent.text]
    print(audioFile)
    
    sound = wave.open(audioFile)
    p = pyaudio.PyAudio()
    chunk = 1024
    stream = p.open(format = p.get_format_from_width(sound.getsampwidth()), channels = sound.getnchannels(), rate = sound.getframerate(), output = True)
    data = sound.readframes(chunk)
    while len(data) > 0:
        stream.write(data)
        data = sound.readframes(chunk)
    stream.stop_stream()
    stream.close()
    
    p.terminate

## Run here:

In [15]:
runLoop()

Enter some text, anything at all...
this is wednesday


/usr/local/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


You approach a scene like that with caution.
PoemsAudio/Andrews/andrews3.wav
